In [10]:
from pyagnps import climate

from pathlib import Path

import pynldas2

In [8]:
clm_annagnps = climate.clm_annagnps_coords(coords=(-89.1, 34.1), start="1980-01-01", end="1990-07-31", date_mode="local")

In [12]:
clm = pynldas2.get_bycoords(coords=(-89.1, 34.1), start_date="1980-01-01", end_date="1990-07-31", source="netcdf")

In [21]:
clm.isna().any().any()

True

In [9]:
root_dir = Path('../../outputs/climate/')

df = clm_annagnps.query_nldas2_generate_annagnps_climate_daily(output_filepath= root_dir / "climate_daily.csv", float_format='%.2f')

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [4]:
df

,Month,Day,Year,Max_Air_Temperature,Min_Air_Temperature,Precip,Dew_Point,Sky_Cover,Wind_Speed,Wind_Direction,Solar_Radiation,Storm_Type_ID,Potential_ET,Actual_ET,Actual_EI,Input_Units_Code
time,,,,,,,,,,,,,,,,
1980-01-01,1,1,1980,9.200000,2.830000,0.0000,2.206102,None,3.603718,114.973831,122.837875,None,2.3140,None,None,1
1980-01-02,1,2,1980,10.050000,1.010000,0.3343,1.780232,None,0.983013,151.945419,84.589043,None,1.3207,None,None,1
1980-01-03,1,3,1980,7.480000,2.440000,10.3941,4.269094,None,1.936419,273.504150,41.893417,None,0.6817,None,None,1
1980-01-04,1,4,1980,6.410000,1.340000,2.3033,1.615022,None,5.365164,137.888397,107.222420,None,2.5982,None,None,1
1980-01-05,1,5,1980,4.740000,0.250000,0.0000,-1.541558,None,3.231326,155.017639,92.897331,None,2.1424,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980-07-27,7,27,1980,30.040001,24.379999,2.2339,23.482895,None,3.180131,105.913399,211.342789,None,5.0532,None,None,1
1980-07-28,7,28,1980,31.820000,23.650000,0.2563,22.864721,None,1.536312,100.881805,294.255005,None,6.7405,None,None,1
1980-07-29,7,29,1980,33.619999,24.440001,0.0000,22.504629,None,2.265506,139.102737,371.034241,None,8.6012,None,None,1


In [5]:
df_station = clm_annagnps.generate_climate_station_file(output_filepath= root_dir / 'climate_station.csv',
                                                        elevation='3dep')

In [6]:
df_station

,Version,Input Units Code,Climate Station Name,Beginning Climate Date 'mm/dd/yyyy',Ending Climate Date 'mm/dd/yyyy',Latitude,Longitude,Elevation,Temperature Lapse Rate,Precipitation N,...,2nd Elevation Rain Factor,2 Yr 24 hr Precipitation,Calibration or Areal Correction Coefficient,Calibration or Areal Correction Exponent,Minimum Interception Evaporation,Maximum Interception Evaporation,Winter Storm Type ID,Spring Storm Type ID,Summer Storm Type ID,Autumn Storm Type ID
0,6.0,1,Station,01/01/1980,07/31/1980,34.1,-89.1,102.48481,,,...,,,,,,,,,,


### Dashboard

In [7]:
import hvplot.pandas
import hvplot

import holoviews as hv
import panel as pn
from panel.widgets import Select, MultiSelect, MultiChoice

columns = ['Max_Air_Temperature', 'Min_Air_Temperature', 'Precip', 'Dew_Point', 'Wind_Speed', 'Wind_Direction', 'Solar_Radiation', 'Potential_ET']

# dropdown = Select(options=columns, sizing_mode='fixed', height=30)
# def plot_timeseries(df, column):
#     return hv.Curve((df.index, df[column])).opts(responsive=True, height=600)

# interactive_plot = pn.bind(plot_timeseries, df, dropdown)

dropdown = MultiChoice(options=columns)
def plot_multi_timeseries(df, columns):
    if columns:
        overlay_list = []
        for col in columns:
            overlay_list.append(hv.Curve((df.index, df[col]))*hv.Scatter((df.index, df[col])))
        return hv.Overlay(overlay_list).opts(responsive=True, height=800)

interactive_plot = pn.bind(plot_multi_timeseries, df, dropdown)

app = pn.Column(dropdown, interactive_plot, sizing_mode="stretch_width")
# app.show(threaded=True)
app.height = 1000
app

# app.save("climate.html", embed=True)


BokehModel(combine_events=True, render_bundle={'docs_json': {'e3e8aea4-7178-4bbc-85de-58cd1a5b777c': {'version…

In [7]:
import py3dep
elev_airmap = py3dep.elevation_bycoords((-89.1, 34.1), source='airmap')
elev_tep = py3dep.elevation_bycoords((-89.1, 34.1), source='tep')
elev_tnm = py3dep.elevation_bycoords((-89.1, 34.1), source='tnm')

In [8]:
print(f"Airmap: {elev_airmap}, TEP: {elev_tep}, TNM: {elev_tnm}")

Airmap: 101, TEP: 102.48480987548828, TNM: 102.227973938


In [15]:
from pathlib import Path

data = {
    'A': 1,
    'B': 'luc',
    'C': "",
    'D': 3.14
}

file = Path('test.csv')

In [18]:
file.write_text(f"{','.join(data.keys())}\n")
with file.open(mode='a') as f:
    f.write(f"{','.join([str(x) for x in data.values()])}")